In [1]:
import numpy as np

In [2]:
import os

In [3]:
read_path = 'raw_data/collect_writer_data/'
step=8
primitive_dim = 7
state_files = os.listdir(read_path+'/state')
states = []
for file in state_files:
    states.append(f'{read_path}state/{file}')
states.sort(key=lambda x:int(x.split('/')[-1][:-4]))

In [4]:
actions = np.load(read_path+'action.npy')
print(actions.shape)

(10200, 3)


In [5]:
size = 10000 - 200*(step-2)
before_x = np.zeros((size,8192,3),dtype=np.float64)
before_v = np.zeros((size,8192,3),dtype=np.float64)
before_F = np.zeros((size,8192,3,3),dtype=np.float64)
before_C = np.zeros((size,8192,3,3),dtype=np.float64)
before_p = np.zeros((size,primitive_dim), dtype=np.float64)
after_x =  np.zeros((size,step-1,8192,3),dtype=np.float64)
action_ =  np.zeros((size,step-1,actions.shape[1]),dtype=np.float64)

In [6]:
cnt = 0
for i in range(10200):
    if i % 51 <= (51-step):
        z = np.load(states[i])
        before_x[cnt] = z['arr_0']
        before_v[cnt] = z['arr_1']
        before_F[cnt] = z['arr_2']
        before_C[cnt] = z['arr_3']
        rest_keys = z.files[4:]
        if len(rest_keys) > 1:
            before_p[cnt] = np.hstack([z[key] for key in rest_keys])
        else:
            before_p[cnt] = z['arr_4']
        for j in range(step-1):
            after_z = np.load(states[i+j+1])
            after_x[cnt,j] = after_z['arr_0']
            action_[cnt,j] = actions[i+j]
        cnt += 1
actions = action_

In [7]:
cnt

8800

In [8]:
sort_index = np.random.choice(size,6000,replace=False)

In [9]:
before_x = before_x[sort_index]
before_v = before_v[sort_index]
before_F = before_F[sort_index]
before_C = before_C[sort_index]
before_p = before_p[sort_index]
after_x = after_x[sort_index]
action = actions[sort_index]

In [10]:

np.savez(
    f'data/writer-v1.npz',
    before_x = before_x,
    before_v = before_v,
    before_F = before_F,
    before_C = before_C,
    before_p = before_p,
    after_x = after_x,
    action = action
)